In [1]:
import pandas as pd
import requests
from datetime import date, timedelta
import json
from pymongo import MongoClient 


In [5]:
client = MongoClient() 
client = MongoClient("mongodb://localhost:27017")

In [6]:
mydatabase = client['Zones'] 
mycollection = mydatabase['Database'] 
myzones = mydatabase['Nifty']

In [7]:
def fiblvlprice(lvl,ind):
    return (explosive['Low'][ind]-explosive['High'][ind])*lvl+explosive['High'][ind]


In [8]:
def wwma(values, n):
    """
     J. Welles Wilder's EMA 
    """
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(df, n=14):
    data = df.copy()
    high = data['high']
    low = data['low']
    close = data['close']
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = wwma(tr, n)
    return atr

In [21]:
def isCandleExplosive(high,low,openn,close,atravg,strength):
    mp = strength
    ratio = 50
    cand = 20
    rang = atravg

    candr = abs(high-low)
    bodyr = abs(openn-close)
    if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):
        return True
    else:
        return False


In [22]:
def checkdata(df,strength=1.5):
    ind = []
    date = []
    o = []
    h = []
    l = []
    c = []
    explosive = pd.DataFrame()

    for i in range(0,len(df['date'])-1):
        if isCandleExplosive(df['high'][i],df['low'][i],df['open'][i],df['close'][i],df['atravg'][i],strength):
            #print(df['date'][i],df['atr'][i])
            ind.append(i)
            date.append(df['date'][i])
            o.append(df['open'][i])
            h.append(df['high'][i])
            l.append(df['low'][i])
            c.append(df['close'][i])
    explosive['index'] = ind
    explosive['Date'] = date
    explosive['Open'] = o
    explosive['High'] = h
    explosive['Low'] = l
    explosive['Close'] = c
    return explosive

In [23]:
def jsonToDf(ticker,collection):
    return pd.read_json((collection.find_one({'ticker':ticker}))['dataframe'])

In [24]:
def putAtrValues(df):
    atri = atr(df,1)
    atr_avg = atr(df,20)
    df = df.merge(atr_avg.rename('atravg'), left_index=True, right_index=True)
    df = df.merge(atri.rename('atr'),left_index=True, right_index=True)
    return df

In [25]:
a = jsonToDf(zone_ticker[2],mycollection)

In [26]:
a = putAtrValues(a)

In [27]:
explosive = checkdata(a,1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6532\3528207061.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):


In [88]:
ltp = 4103.8

In [28]:
dmzs = []
def getDmzs(explosive,df,ticker):
    obj = []
    for i in range(0,len(explosive['Date'])-1):
        base = explosive['index'][i]-1
        if explosive['Close'][i] > explosive['Open'][i] and abs(df['close'][base]-df['open'][base]) <= (explosive['High'][i]-explosive['Low'][i])*0.2 and explosive['Close'][i] > fiblvlprice(0.4,i) and df['open'][base] < fiblvlprice(0.5,i) and df['close'][base] < fiblvlprice(0.5,i) and explosive['Open'][i] < fiblvlprice(0.5,i) and df['high'][base] < fiblvlprice(0.5,i):
            obj.append(
                {
                    "l":1,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            #epm and open[1] > open and close[1] < low and open[1] < fibprice(0.75) and high-low > high[1]-low[1]
        elif explosive['Close'][i] > explosive['Open'][i] and df['open'][base] > explosive['Open'][i] and df['close'][base] < explosive['Low'][i] and df['open'][base] < fiblvlprice(0.75,i) and explosive['High'][i]-explosive['Low'][i] > df['high'][base]-df['low'][base] and (explosive['Close'][i]-explosive['Close'][i])*0.25 > (df['open'][base]-df['close'][base]):
            obj.append(
                {
                    "l":2,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.25,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.4,i):
            obj.append(
                {
                    "l":3,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.3,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.2,i):
            obj.append(
                {
                    "l":4,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            
    dmzs.append({ticker:obj})

In [29]:
getDmzs(explosive,a,zone_ticker[2])

NameError: name 'ltp' is not defined

In [20]:
dmzs

[]

In [54]:
jstock = mycollection.find({})

In [151]:
stock = zone_ticker

In [105]:
dmzs = []
for i in range(0,len(stock)):
    a = jsonToDf(stock[i],mycollection)
    a = putAtrValues(a)
    explosive = checkdata(a)
    getDmzs(explosive,a,stock[i])

C:\Users\Admin\AppData\Local\Temp\ipykernel_12020\2823544080.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):


In [110]:
stock[0]

'ACC'

In [153]:
zones

[{'ticker': 'ACC', 'zone': []},
 {'ticker': 'ADANIGREEN',
  'zone': [{'l': 1,
    'legoutDate': Timestamp('2024-02-15 09:15:00+0530', tz='pytz.FixedOffset(330)'),
    'baseHigh': 1859.85,
    'baseLow': 1845.0,
    'dist': -5.2547121752419805,
    'ltp': 1963.0,
    'chartData': {'series': [{'data': [{'x': Timestamp('2024-02-14 15:15:00+0530', tz='pytz.FixedOffset(330)'),
         'y': [1850.95, 1859.85, 1845.0, 1853.8]},
        {'x': Timestamp('2024-02-15 09:15:00+0530', tz='pytz.FixedOffset(330)'),
         'y': [1868.0, 1959.7, 1857.15, 1921.95]}]}],
     'legoutDate': Timestamp('2024-02-15 09:15:00+0530', tz='pytz.FixedOffset(330)')}},
   {'l': 1,
    'legoutDate': Timestamp('2024-02-07 09:15:00+0530', tz='pytz.FixedOffset(330)'),
    'baseHigh': 1723.0,
    'baseLow': 1716.05,
    'dist': -12.226184411614875,
    'ltp': 1963.0,
    'chartData': {'series': [{'data': [{'x': Timestamp('2024-02-06 15:15:00+0530', tz='pytz.FixedOffset(330)'),
         'y': [1720.7, 1723.0, 1716.05, 17

In [180]:
jstock

# MAIN CODE

In [221]:
def updateSettings():
    zone_ticker = []
    for i in myzones.find({}):
        zone_ticker.append(i['ticker'])

In [14]:
zone_ticker = []
for i in myzones.find({}):
    zone_ticker.append(i['ticker'])
print(zone_ticker)

['ABB', 'ACC', 'ADANIGREEN', 'AARTIIND', 'AMBUJACEM']


In [182]:
zone_ticker = ['ACC', 'ADANIGREEN', 'AARTIIND', 'AMBUJACEM']

In [239]:
getDmzs()

4498.0
ABB


KeyError: 43

In [30]:
dmzs = []
for j in range(0,len(zone_ticker)):
    df = jsonToDf(zone_ticker[j],mycollection)
    ltp = df['close'].iloc[-1]
    print(ltp)
    print(zone_ticker[j])
    df = putAtrValues(df)
    explosive1 = checkdata(df,1)
    explosive = explosive1.iloc[::-1].reset_index(drop=True)
    #print(explosive)
    obj = []
    for i in range(0,len(explosive['Date'])-1):
        base = explosive['index'][i]-1
        if explosive['Close'][i] > explosive['Open'][i] and abs(df['close'][base]-df['open'][base]) <= (explosive['High'][i]-explosive['Low'][i])*0.2 and explosive['Close'][i] > fiblvlprice(0.7,i) and df['open'][base] < fiblvlprice(0.5,i) and df['close'][base] < fiblvlprice(0.5,i) and explosive['Open'][i] < fiblvlprice(0.3,i) and df['high'][base] < fiblvlprice(0.5,i):
            obj.append(
                {
                    "l":1,
                        "legoutDate":explosive['Date'][i],
                        "baseHigh":df['high'][base],
                        "baseLow":df['low'][base],
                        "dist":((df['high'][base]-ltp)/ltp)*100,
                        "ltp":ltp,
                        "chartData":{
                            "series":[
                                {
                                    "data":
                                    [
                                            {
                                    "x":df['date'][base],
                                    "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                                },
                                {
                                    "x":explosive['Date'][i],
                                    "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                                }
                                    ]
                                }
                            ],
                            "legoutDate":explosive['Date'][i]
                        }
                    })
                #epm and open[1] > open and close[1] < low and open[1] < fibprice(0.75) and high-low > high[1]-low[1]
        elif explosive['Close'][i] > explosive['Open'][i] and df['open'][base] > explosive['Open'][i] and df['close'][base] < explosive['Low'][i] and df['open'][base] < fiblvlprice(0.75,i) and explosive['High'][i]-explosive['Low'][i] > df['high'][base]-df['low'][base] and (explosive['Close'][i]-explosive['Close'][i])*0.25 > (df['open'][base]-df['close'][base]):
            obj.append(
                    {
                        "l":2,
                        "legoutDate":explosive['Date'][i],
                        "baseHigh":df['high'][base],
                        "baseLow":df['low'][base],
                        "dist":((df['high'][base]-ltp)/ltp)*100,
                        "ltp":ltp,
                        "chartData":{
                            "series":[
                                {
                                    "data":
                                    [
                                            {
                                    "x":df['date'][base],
                                    "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                                },
                                {
                                    "x":explosive['Date'][i],
                                    "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                                }
                                    ]
                                }
                            ],
                            "legoutDate":explosive['Date'][i]
                        }
                    })
                #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.25,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.4,i):
            obj.append(
                    {
                        "l":3,
                        "legoutDate":explosive['Date'][i],
                        "baseHigh":df['high'][base],
                        "baseLow":df['low'][base],
                        "dist":((df['high'][base]-ltp)/ltp)*100,
                        "ltp":ltp,
                        "chartData":{
                            "series":[
                                {
                                    "data":
                                    [
                                            {
                                    "x":df['date'][base],
                                    "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                                },
                                {
                                    "x":explosive['Date'][i],
                                    "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                                }
                                    ]
                                }
                            ],
                            "legoutDate":explosive['Date'][i]
                        }
                    })
                #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.3,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.2,i):
            obj.append(
                    {
                        "l":4,
                        "legoutDate":explosive['Date'][i],
                        "baseHigh":df['high'][base],
                        "baseLow":df['low'][base],
                        "dist":((df['high'][base]-ltp)/ltp)*100,
                        "ltp":ltp,
                        "chartData":{
                            "series":[
                                {
                                    "data":
                                    [
                                            {
                                    "x":df['date'][base],
                                    "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                                },
                                {
                                    "x":explosive['Date'][i],
                                    "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                                }
                                    ]
                                }
                            ],
                            "legoutDate":explosive['Date'][i]
                        }
                    })

    dmzs.append({zone_ticker[j]:obj})

4498.0
ABB
2672.05
ACC
1963.0
ADANIGREEN
689.3
AARTIIND
584.4
AMBUJACEM


C:\Users\Admin\AppData\Local\Temp\ipykernel_6532\3528207061.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):


In [32]:
zones = []
#b=['TCS']
for ti in range(0,len(zone_ticker)):
    brokenzone = []
    df = jsonToDf(zone_ticker[ti],mycollection)
    #print(stock[ti])
    for i in range (0,len(dmzs[ti][zone_ticker[ti]])):
        for j in range(0,len(df)):
            #print(type(df['date'][j]))
            if df['date'][j] == dmzs[ti][zone_ticker[ti]][i]['chartData']['legoutDate']:
                #print(df['date'][j])
                for k in range(j+1,len(df)):
                    #print(k,df['date'][k],df['low'][k],dmzs[0][tiks[0]][i]['baseLow'])
                    if dmzs[ti][zone_ticker[ti]][i]['baseLow'] > df['low'][k]:
                        #f0e0rti,dmzs[ti][tiks[ti]][i]['legoutDate'],'broken',dmzs[ti][tiks[ti]][i]['baseLow'],df['date'][k])
                        brokenzone.append(dmzs[ti][zone_ticker[ti]][i]['chartData']['legoutDate'])
                        break
    a = dmzs[ti][zone_ticker[ti]]
    bz_set = set(brokenzone)
    #print(tiks[ti])
    # Efficiently remove matching dictionaries from the end
    for i in range(len(a) - 1, -1, -1):
        if a[i]['legoutDate'] in bz_set:
            del a[i]
    #print(json.dumps(a,indent=4))
    zones.append({'ticker':zone_ticker[ti],'zone':a})

In [33]:
len(zones[1]['zone'])

10

# End of MainCOde

In [192]:
import http.server
import json

HOST = 'localhost'  # Replace with your desired hostname or IP address
PORT = 8000  # Replace with your desired port number

class MyRequestHandler(http.server.BaseHTTPRequestHandler):
    def do_POST(self):
        if self.path == '/gzabc' or self.path == '/gzabd':
            # Access request data
            content_length = int(self.headers.get('Content-Length', 0))
            request_body = self.rfile.read(content_length).decode('utf-8')

            # Process or validate the request body (as needed)
            processed_data = process_or_validate_data(request_body)  # Implement your processing logic here

            # Prepare response
            response_body = str(zones)
            self.send_response(200)
            self.send_header('Content-Type', 'application/json')
            self.end_headers()
            self.wfile.write(response_body.encode('utf-8'))
        else:
            self.send_error(404, 'Not Found')

def process_or_validate_data(data):
    # Add your data processing or validation logic here
    # Example:
    try:
        return json.loads(data)
    except json.JSONDecodeError:
        return None

with http.server.HTTPServer((HOST, PORT), MyRequestHandler) as httpd:
    print(f"Serving at http://{HOST}:{PORT}")
    httpd.serve_forever()


Serving at http://localhost:8000


127.0.0.1 - - [20/Feb/2024 10:14:20] "POST /gzabc HTTP/1.1" 200 -


KeyboardInterrupt: 

# Where am i Stuck
# 1) demand zones are not identified when changing strenght level